In [ ]:
import sys
from pathlib import Path

# Jupyter Notebook 环境下的路径处理
try:
    root_path = next(
        (
            p
            for p in Path(__file__).resolve().parents
            if (p / "pyproject.toml").is_file()
        ),
        None,
    )
except NameError:
    current_dir = Path.cwd()
    root_path = next(
        (
            p
            for p in [current_dir] + list(current_dir.parents)
            if (p / "pyproject.toml").is_file()
        ),
        None,
    )

if root_path:
    sys.path.insert(0, str(root_path))

In [ ]:
from py_entry.runner.params import FormatResultsConfig, DiagnoseStatesConfig
from py_entry.Test.backtest.strategies import get_strategy
from py_entry.runner import Backtest

strategy = get_strategy("reversal_extreme")

runner = Backtest(
    data_source=strategy.data_config,
    indicators=strategy.indicators_params,
    signal=strategy.signal_params,
    backtest=strategy.backtest_params,
    signal_template=strategy.signal_template,
    engine_settings=strategy.engine_settings,
)
result = runner.run()

result.format_for_export(FormatResultsConfig(dataframe_format="csv"))
assert result.summary is not None, "检测到None"

In [ ]:
# import polars as pl
# pl.Config.set_tbl_rows(500)
result.summary.backtest_result

In [ ]:
# DiagnoseStatesConfig usage needs update or removal if not supported
# from py_entry.runner.params import DiagnoseStatesConfig
# result.diagnose_states(...) - Not directly available on RunResult yet.
pass

In [ ]:
from py_entry.io import DisplayConfig, DashboardOverride

config = DisplayConfig(
    embed_data=False,
    width="100%",
    aspect_ratio="16/9",
    override=DashboardOverride(
        show=["0,0,0,1"],
        showInLegend=["0,0,0,1"],
        showRiskLegend="1,1,1,1",
        showLegendInAll=True,
    ).to_dict(),
)

result.display(config=config)

In [ ]:
import json
from pathlib import Path

json_path = Path(Path.cwd() / "performance.json")
with open(json_path, "w") as f:
    json.dump(result.summary.performance, f, indent=2, sort_keys=True)
print(f"✅ 已保存 performance.json: {json_path}")